# Text-to-SQL 데모

자연어 질문을 SQL 쿼리로 변환하여 데이터베이스를 조회하는 실습입니다.

## 학습 목표

- SQLite 데이터베이스 생성 및 데이터 삽입
- 테이블 스키마 정보를 프롬프트에 활용
- EXAONE 3.5를 사용한 자연어 → SQL 변환
- SQL 쿼리 실행 및 결과 확인

## 1. 필요한 라이브러리 설치

In [ ]:
# 필요한 경우에만 설치
# !pip install transformers torch pandas

In [ ]:
import os
import sqlite3
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

print("✓ 라이브러리 로드 완료")

## 2. EXAONE 3.5 모델 로드

Day 1에서 사용한 EXAONE 3.5 Instruct 모델을 불러옵니다.

In [ ]:
print("🤖 EXAONE 3.5 모델 로딩 중... (시간이 걸릴 수 있습니다)\n")

# Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    trust_remote_code=True
)

# Model 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map='auto'
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✓ EXAONE 3.5 모델 로드 완료 (Device: {device})")

## 3. SQLite 데이터베이스 생성

간단한 **직원(employees)** 테이블을 만들어 샘플 데이터를 넣겠습니다.

### 테이블 구조
- `id`: 직원 ID (정수, 기본키)
- `name`: 이름 (텍스트)
- `department`: 부서 (텍스트)
- `position`: 직급 (텍스트)
- `salary`: 연봉 (정수, 만원 단위)
- `hire_date`: 입사일 (텍스트)

In [ ]:
# 데이터베이스 연결 (파일이 없으면 자동 생성)
conn = sqlite3.connect('company.db')
cursor = conn.cursor()

# 기존 테이블이 있으면 삭제
cursor.execute('DROP TABLE IF EXISTS employees')

# 테이블 생성
cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department TEXT NOT NULL,
    position TEXT NOT NULL,
    salary INTEGER NOT NULL,
    hire_date TEXT NOT NULL
)
''')

print("✓ employees 테이블 생성 완료")

## 4. 샘플 데이터 삽입

다양한 부서와 직급의 직원 데이터를 넣어서 의미 있는 쿼리를 테스트할 수 있게 합니다.

In [ ]:
# 샘플 직원 데이터
employees_data = [
    (1, '김철수', '개발팀', '시니어 개발자', 7000, '2020-03-15'),
    (2, '이영희', '개발팀', '주니어 개발자', 4500, '2022-06-01'),
    (3, '박민수', '영업팀', '영업 팀장', 8000, '2018-01-10'),
    (4, '최지원', '영업팀', '영업 사원', 4000, '2023-02-20'),
    (5, '정수진', '인사팀', '인사 매니저', 6500, '2019-05-12'),
    (6, '강동훈', '개발팀', '팀장', 9000, '2017-08-25'),
    (7, '윤서연', '마케팅팀', '마케터', 5000, '2021-11-03'),
    (8, '조민준', '마케팅팀', '마케팅 팀장', 7500, '2019-09-18'),
]

# 데이터 삽입
cursor.executemany(
    'INSERT INTO employees (id, name, department, position, salary, hire_date) VALUES (?, ?, ?, ?, ?, ?)',
    employees_data
)

# 변경사항 저장
conn.commit()

print(f"✓ {len(employees_data)}명의 직원 데이터 삽입 완료")

## 5. 데이터 확인

제대로 들어갔는지 전체 데이터를 조회해봅니다.

In [ ]:
# 전체 데이터 조회
df = pd.read_sql_query('SELECT * FROM employees', conn)
print("📊 전체 직원 데이터:\n")
display(df)

## 6. 테이블 스키마 정보 가져오기

LLM이 정확한 SQL을 생성하려면 **테이블 구조**를 알아야 합니다.

- 테이블 이름
- 컬럼 이름과 타입
- 각 컬럼의 의미

In [ ]:
# 테이블 스키마 조회
cursor.execute("PRAGMA table_info(employees)")
schema = cursor.fetchall()

print("📋 employees 테이블 스키마:\n")
for col in schema:
    print(f"  - {col[1]} ({col[2]})")

## 7. 스키마 설명 문자열 생성

LLM에게 줄 프롬프트에 포함할 상세한 테이블 설명을 만듭니다.

In [ ]:
# 스키마 설명 문자열
schema_description = """
데이터베이스: company.db (SQLite)

테이블: employees
설명: 회사 직원 정보를 저장하는 테이블

컬럼:
- id: 직원 고유 ID (INTEGER, PRIMARY KEY)
- name: 직원 이름 (TEXT)
- department: 소속 부서 (TEXT) - 예: '개발팀', '영업팀', '인사팀', '마케팅팀'
- position: 직급 (TEXT) - 예: '팀장', '시니어 개발자', '주니어 개발자', '사원'
- salary: 연봉 (INTEGER, 만원 단위) - 예: 7000은 7천만원
- hire_date: 입사일 (TEXT, 'YYYY-MM-DD' 형식)

샘플 데이터:
- 총 8명의 직원
- 부서: 개발팀(3명), 영업팀(2명), 인사팀(1명), 마케팅팀(2명)
- 연봉 범위: 4000만원 ~ 9000만원
"""

print(schema_description)

## 8. Text-to-SQL 프롬프트 템플릿

자연어 질문을 SQL로 변환하는 프롬프트를 만듭니다.

### 핵심 요소
1. **역할 부여**: SQL 전문가 역할
2. **스키마 제공**: 테이블 구조 상세 설명
3. **제약 조건**: SQLite 문법 준수, SELECT만 허용
4. **출력 형식**: SQL 쿼리만 반환

In [ ]:
def create_text2sql_prompt(question: str, schema: str) -> str:
    """
    자연어 질문을 SQL로 변환하는 프롬프트를 생성합니다.
    
    Args:
        question: 사용자의 자연어 질문
        schema: 데이터베이스 스키마 설명
    
    Returns:
        LLM에게 전달할 프롬프트 문자열
    """
    prompt = f"""당신은 SQL 전문가입니다. 사용자의 자연어 질문을 SQLite 쿼리로 변환해주세요.

### 데이터베이스 스키마
{schema}

### 규칙
1. SQLite 문법을 사용하세요
2. SELECT 쿼리만 작성하세요 (INSERT, UPDATE, DELETE 금지)
3. 컬럼명과 테이블명을 정확히 사용하세요
4. 한글 조건은 LIKE를 사용하세요
5. 주석이나 설명 없이 SQL 쿼리만 반환하세요
6. 코드 블록(```)을 사용하지 마세요

### 사용자 질문
{question}

### SQL 쿼리
"""
    return prompt

# 테스트
test_question = "개발팀 직원들의 이름과 연봉을 보여줘"
test_prompt = create_text2sql_prompt(test_question, schema_description)
print("📝 생성된 프롬프트 예시:\n")
print(test_prompt)

## 9. EXAONE으로 SQL 생성 함수

프롬프트를 EXAONE 3.5에 전달하여 SQL 쿼리를 생성합니다.

In [ ]:
def generate_sql(question: str) -> str:
    """
    자연어 질문을 SQL 쿼리로 변환합니다.
    
    Args:
        question: 사용자의 자연어 질문
    
    Returns:
        생성된 SQL 쿼리 문자열
    """
    # 프롬프트 생성
    prompt = create_text2sql_prompt(question, schema_description)
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_length = inputs.input_ids.shape[1]
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.1,  # 일관된 결과를 위해 낮게 설정
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode
    generated_tokens = outputs[0][input_length:]
    sql_query = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # 코드 블록 제거 (```sql ... ``` 형태로 올 수 있음)
    if '```' in sql_query:
        # 코드 블록 안의 내용만 추출
        import re
        match = re.search(r'```(?:sql)?\s*([^`]+)```', sql_query)
        if match:
            sql_query = match.group(1).strip()
    
    # 첫 번째 세미콜론까지만 (설명이 붙는 경우 대비)
    if ';' in sql_query:
        sql_query = sql_query.split(';')[0] + ';'
    
    # 개행 정리
    sql_query = ' '.join(sql_query.split())
    
    return sql_query

print("✓ SQL 생성 함수 정의 완료")

## 10. SQL 실행 함수

생성된 SQL을 실제로 실행하여 결과를 가져옵니다.

In [ ]:
def execute_sql(sql_query: str) -> pd.DataFrame:
    """
    SQL 쿼리를 실행하고 결과를 DataFrame으로 반환합니다.
    
    Args:
        sql_query: 실행할 SQL 쿼리
    
    Returns:
        쿼리 결과를 담은 DataFrame
    """
    try:
        df = pd.read_sql_query(sql_query, conn)
        return df
    except Exception as e:
        print(f"❌ SQL 실행 오류: {e}")
        return pd.DataFrame()

print("✓ SQL 실행 함수 정의 완료")

## 11. Text-to-SQL 전체 파이프라인

자연어 질문 → SQL 생성 → 실행 → 결과 반환을 한 번에 처리합니다.

In [ ]:
def text_to_sql_pipeline(question: str):
    """
    자연어 질문을 받아 SQL로 변환하고 실행하여 결과를 보여줍니다.
    
    Args:
        question: 사용자의 자연어 질문
    """
    print("🔍 질문:", question)
    print("\n⚙️  SQL 생성 중...\n")
    
    # SQL 생성
    sql_query = generate_sql(question)
    print("📝 생성된 SQL:")
    print(f"```sql\n{sql_query}\n```\n")
    
    # SQL 실행
    print("🚀 SQL 실행 중...\n")
    result_df = execute_sql(sql_query)
    
    # 결과 출력
    if not result_df.empty:
        print("✅ 결과:")
        display(result_df)
        print(f"\n📊 총 {len(result_df)}개 행")
    else:
        print("❌ 결과가 없거나 오류가 발생했습니다.")
    
    print("\n" + "="*60 + "\n")

print("✓ Text-to-SQL 파이프라인 함수 정의 완료")

## 12. 실습 예제

다양한 자연어 질문을 SQL로 변환하여 실행해봅니다.

### 예제 1: 단순 조회

특정 부서의 직원 정보를 조회합니다.

In [ ]:
text_to_sql_pipeline("개발팀 직원들의 이름과 연봉을 보여줘")

### 예제 2: 집계 함수

부서별 평균 연봉을 계산합니다.

In [ ]:
text_to_sql_pipeline("부서별 평균 연봉을 알려줘")

### 예제 3: 정렬

연봉이 높은 순서로 직원을 정렬합니다.

In [ ]:
text_to_sql_pipeline("연봉이 가장 높은 직원 3명을 보여줘")

### 예제 4: 조건 필터링

연봉 범위로 직원을 필터링합니다.

In [ ]:
text_to_sql_pipeline("연봉이 6000만원 이상인 직원들을 알려줘")

### 예제 5: 날짜 조건

입사일 기준으로 필터링합니다.

In [ ]:
text_to_sql_pipeline("2020년 이후에 입사한 직원들의 이름과 입사일을 보여줘")

### 예제 6: 복잡한 조건

여러 조건을 조합합니다.

In [ ]:
text_to_sql_pipeline("개발팀에서 연봉이 5000만원 이상인 직원들을 보여줘")

### 예제 7: COUNT

부서별 직원 수를 계산합니다.

In [ ]:
text_to_sql_pipeline("각 부서에 직원이 몇 명씩 있는지 알려줘")

## 13. 직접 질문해보기

여러분의 질문을 입력해보세요!

In [ ]:
# 여기에 자유롭게 질문을 입력하세요
my_question = "팀장 직급을 가진 사람들의 이름과 부서를 보여줘"

text_to_sql_pipeline(my_question)

## 14. 정리 및 연결 종료

In [ ]:
# 데이터베이스 연결 종료
conn.close()
print("✓ 데이터베이스 연결 종료")

## 핵심 정리

### Text-to-SQL의 장점
1. **접근성**: SQL을 모르는 사용자도 데이터 조회 가능
2. **생산성**: 복잡한 쿼리를 자연어로 간단히 요청
3. **유연성**: 다양한 질문 형태 지원

### 핵심 요소
1. **스키마 정보**: 테이블 구조와 컬럼 의미를 명확히 제공
2. **프롬프트 엔지니어링**: 역할, 제약조건, 출력 형식 지정
3. **오류 처리**: SQL 실행 시 예외 처리
4. **EXAONE 3.5 활용**: 한국어 질문에 특화된 모델 사용

### 실전 활용 시 고려사항
1. **보안**: SELECT만 허용, 파라미터 검증
2. **성능**: 복잡한 쿼리는 타임아웃 설정
3. **검증**: 생성된 SQL을 사용자에게 확인받기
4. **로깅**: 쿼리 히스토리 저장 및 분석
5. **후처리**: 생성된 SQL 정제 (코드 블록 제거, 불필요한 설명 제거)